# Run Synsat for ICON Data

Here, we will 
- load ICON test data (limited area simulation, stored in separate files, from IFCES2 project)
- run Synsat on ICON and
- finally store the Synsats into a netcdf file

The workflow is similar to [02-Run-Synsat-for-ERA5-Data.ipynb](02-Run-Synsat-for-ERA5-Data.ipynb)

## Setup Env and Load Libraries 

In [1]:
import os, sys
os.environ['RTTOV_PYTHON_WRAPPER'] = '/work/bb1262/tools/rttov/rttov-v13.2/wrapper'

import warnings
warnings.filterwarnings("ignore")

In [2]:
import synsatipy

from synsatipy.synsat import SynSat

import synsatipy.synsat_example_data as synsat_example_data

no git hash can be obtained


## Steps towards Synsat 

### Step I: Initialize Synsat class

In [3]:
s = SynSat( )

... [synsat] set cloud / aerosol file to  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/cldaer_visir/sccldcoef_msg_3_seviri.dat
... [synsat] load coefficient file /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/rttov13pred54L/rtcoef_msg_3_seviri_o3.dat


 2024/05/05  13:05:16  Load coefficients:
Load successful >>>>> inst_id : 1, nchannels : 6.
 2024/05/05  13:05:16  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/rttov13pred54L/rtcoef_msg_3_seviri_o3.dat
 2024/05/05  13:05:16  /work/bb1262/tools/rttov/rttov-v13.2/rtcoef_rttov13/cldaer_visir/sccldcoef_msg_3_seviri.dat


### Step II: Load Example Data 

In [4]:
iconname = synsat_example_data.get_example_data( 'icon02' )
s.load( iconname )

... [synsat] read data from file  /work/bb1376/data/icon/atlantic-cases/paulette/ifces2-atlanXL-20200907-exp021/POSTPROC//3d_full_base_DOM02_ML_20200912T000000Z_regrid7km.nc


The ICON data have been loaded into a data handler. You can inspect the data as follows:

In [5]:
icon = s.synsat.data_handler.input_data

In [6]:
print(icon)

<xarray.Dataset> Size: 2GB
Dimensions:  (lon: 1094, lat: 767, lev: 70, time: 1)
Coordinates:
  * lon      (lon) float32 4kB -85.0 -84.94 -84.87 ... -15.18 -15.11 -15.05
  * lat      (lat) float32 3kB 0.0 0.06 0.12 0.18 ... 45.78 45.84 45.9 45.96
  * height   (lev) float64 560B 1.0 2.0 3.0 4.0 5.0 ... 66.0 67.0 68.0 69.0 70.0
  * time     (time) datetime64[ns] 8B 2020-09-12
Dimensions without coordinates: lev
Data variables:
    p        (time, lev, lat, lon) float32 235MB dask.array<chunksize=(1, 39, 767, 1094), meta=np.ndarray>
    t        (time, lev, lat, lon) float32 235MB dask.array<chunksize=(1, 39, 767, 1094), meta=np.ndarray>
    q        (time, lev, lat, lon) float32 235MB dask.array<chunksize=(1, 39, 767, 1094), meta=np.ndarray>
    clwc     (time, lev, lat, lon) float32 235MB dask.array<chunksize=(1, 39, 767, 1094), meta=np.ndarray>
    ciwc     (time, lev, lat, lon) float32 235MB dask.array<chunksize=(1, 39, 767, 1094), meta=np.ndarray>
    cswc     (time, lev, lat, lon) fl

The actual ICON data are not really small. At this time, however, the data are not loaded into memory. 

Maybe this is really too large for our small example here. **Let's redo the input and subset the ICON data!**


### Step IIb: Subset and Thean Load Again Example Data 

In [7]:
iconname = synsat_example_data.get_example_data( 'icon01' )
s.load( iconname, isel = {'lon': slice(0,None,4), 'lat': slice(0,None, 4)} )

... [synsat] read data from file  /work/bb1376/data/icon/atlantic-cases/paulette/ifces2-atlanXL-20200907-exp021/POSTPROC//3d_full_base_DOM01_ML_20200912T000000Z_regrid7km.nc


The ICON data have been loaded into a data handler. You can inspect the data as follows:

In [8]:
icon = s.synsat.data_handler.input_data

In [9]:
print(icon)

<xarray.Dataset> Size: 104MB
Dimensions:  (lon: 274, lat: 192, lev: 70, time: 1)
Coordinates:
  * lon      (lon) float32 1kB -85.0 -84.74 -84.49 ... -15.62 -15.37 -15.11
  * lat      (lat) float32 768B 0.0 0.24 0.48 0.72 ... 45.12 45.36 45.6 45.84
  * height   (lev) float64 560B 1.0 2.0 3.0 4.0 5.0 ... 66.0 67.0 68.0 69.0 70.0
  * time     (time) datetime64[ns] 8B 2020-09-12
Dimensions without coordinates: lev
Data variables:
    p        (time, lev, lat, lon) float32 15MB dask.array<chunksize=(1, 39, 192, 274), meta=np.ndarray>
    t        (time, lev, lat, lon) float32 15MB dask.array<chunksize=(1, 39, 192, 274), meta=np.ndarray>
    q        (time, lev, lat, lon) float32 15MB dask.array<chunksize=(1, 39, 192, 274), meta=np.ndarray>
    clwc     (time, lev, lat, lon) float32 15MB dask.array<chunksize=(1, 39, 192, 274), meta=np.ndarray>
    ciwc     (time, lev, lat, lon) float32 15MB dask.array<chunksize=(1, 39, 192, 274), meta=np.ndarray>
    cswc     (time, lev, lat, lon) float32 15

Yes, much better and smaller now!

### Step III: Setup Run Options and Start Execution 

In [10]:
s._options.Nthreads=4
s._options.NprofsPerCall = 4000

In [11]:
%%time

s.run( chunked = True )

... [synsat] running chunk {'profile': slice(0, 4000, None)}


IR emissivity atlas loaded successfully
Atlas deallocated.
 2024/05/05  13:05:24  Running RTTOV using nthreads =    4 and nprofs_per_call =     4000


... [synsat] running chunk {'profile': slice(4000, 8000, None)}
... [synsat] atlasses are already loaded


 2024/05/05  13:05:28  Running RTTOV using nthreads =    4 and nprofs_per_call =     4000


... [synsat] running chunk {'profile': slice(8000, 12000, None)}
... [synsat] atlasses are already loaded


...

... [synsat] running chunk {'profile': slice(48000, 52000, None)}
... [synsat] atlasses are already loaded


 2024/05/05  13:06:10  Running RTTOV using nthreads =    4 and nprofs_per_call =     4000


... [synsat] running chunk {'profile': slice(52000, None, None)}


IR emissivity atlas loaded successfully
Atlas deallocated.
 2024/05/05  13:06:14  Running RTTOV using nthreads =    4 and nprofs_per_call =     4000


CPU times: user 1min 8s, sys: 42.1 s, total: 1min 51s
Wall time: 57.3 s


### Step IV: Extract the Output

In [12]:
icon_synsat = s.extract_output()

In [13]:
print( icon_synsat )

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 1, lon: 274, lat: 192)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-09-12
  * lon      (lon) float32 1kB -85.0 -84.74 -84.49 ... -15.62 -15.37 -15.11
  * lat      (lat) float32 768B 0.0 0.24 0.48 0.72 ... 45.12 45.36 45.6 45.84
Data variables:
    bt062    (time, lon, lat) float64 421kB 226.2 226.2 226.1 ... 233.6 232.2
    bt073    (time, lon, lat) float64 421kB 244.4 244.3 244.2 ... 257.2 256.5
    bt087    (time, lon, lat) float64 421kB 278.9 278.9 278.8 ... 285.9 286.0
    bt108    (time, lon, lat) float64 421kB 282.8 282.7 282.6 ... 287.9 288.1
    bt120    (time, lon, lat) float64 421kB 280.9 280.8 280.8 ... 287.1 287.4
    bt134    (time, lon, lat) float64 421kB 252.2 252.1 252.1 ... 264.1 264.3
Attributes:
    author:                Fabian Senf
    contact:               senf@tropos.de
    institution:           Leibniz Institute for Tropospheric Research
    creation_time:         2024-05-05 13:06:15.126531
    synsat_v

### Step V: Store Data 

In [14]:
outdir = './Data'

if not os.path.isdir( outdir ):
    os.makedirs( outdir )
outfile = f'{outdir}/synsat_icon_example_data.nc'

print(f'... store synsat at {outfile}')
icon_synsat.to_netcdf(outfile)


... store synsat at ./Data/synsat_icon_example_data.nc
